# Preprocessing Play Market Reviews

This notebook downloads the dataset, cleans it using vectorized operations, computes product-level scores, and upserts the data into ChromaDB.
It merges both Apps and Games datasets into a single unified `product_id` schema.

### Notes
**Preprocessing:**
1. URLs are removed, but HTML tags are not strictly removed since Play Store reviews are plain text
2. Whitespace normalization: new lines, tabs, multiple spaces are collapsed to single spaces
Special characters: emojis and punctuation are kept for semantic search and meaning
3. Case normalization: text kept as True Case

## 1. Imports

In [1]:
import re
import os
import math
import numpy as np
import pandas as pd
from pathlib import Path
import kagglehub
import chromadb

/Users/chenchenliu/sentio-plus/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET = 'dmytrobuhai/play-market-2025-1m-reviews-500-titles'
REVIEW_CSVS = ['apps_info.csv', 'apps_reviews.csv', 'games_info.csv', 'games_reviews.csv']

## 2. Download Dataset

In [3]:
# Download latest version
path = kagglehub.dataset_download(DATASET)
print("Path to dataset files:", path)

data_dir = Path(path)
print("Path to dataset files:", path)

Path to dataset files: /Users/chenchenliu/.cache/kagglehub/datasets/dmytrobuhai/play-market-2025-1m-reviews-500-titles/versions/1
Path to dataset files: /Users/chenchenliu/.cache/kagglehub/datasets/dmytrobuhai/play-market-2025-1m-reviews-500-titles/versions/1


## 3. List files in the download folder

In [4]:
if 'data_dir' not in locals():
    print("Error: data_dir is not defined. Please run the previous cell.")
else:
    files = [p for p in data_dir.rglob("*") if p.is_file()]
    print(f"Found {len(files)} files:")
    for p in files[:50]:
        print(p)

Found 4 files:
/Users/chenchenliu/.cache/kagglehub/datasets/dmytrobuhai/play-market-2025-1m-reviews-500-titles/versions/1/games_reviews.csv
/Users/chenchenliu/.cache/kagglehub/datasets/dmytrobuhai/play-market-2025-1m-reviews-500-titles/versions/1/apps_info.csv
/Users/chenchenliu/.cache/kagglehub/datasets/dmytrobuhai/play-market-2025-1m-reviews-500-titles/versions/1/games_info.csv
/Users/chenchenliu/.cache/kagglehub/datasets/dmytrobuhai/play-market-2025-1m-reviews-500-titles/versions/1/apps_reviews.csv


## 4. Load & Merge Datasets (Apps + Games)

In [5]:
def load_and_standardize(reviews_path, info_path, id_col, name_col):
    """
    Loads reviews and info, merges them, and standardizes columns to product_id/product_name.
    """
    print(f"Loading {reviews_path.name}...")
    revs = pd.read_csv(reviews_path)
    revs = revs.rename(columns={id_col: "product_id"})

    print(f"Loading {info_path.name}...")
    info = pd.read_csv(info_path)
    info = info.rename(columns={id_col: "product_id", name_col: "product_name"})
    
    # Merge
    merged = pd.merge(revs, info, on="product_id", how="left")
    return merged

# Find files mapping
# We look for specific filenames known in this dataset
file_map = {f.name: f for f in files}

parts = []

# 1. Process Games
if "games_reviews.csv" in file_map and "games_info.csv" in file_map:
    games = load_and_standardize(
        file_map["games_reviews.csv"],
        file_map["games_info.csv"],
        id_col="game_id",
        name_col="game_name"
    )
    games["type"] = "game"
    parts.append(games)

# 2. Process Apps
if "apps_reviews.csv" in file_map and "apps_info.csv" in file_map:
    apps = load_and_standardize(
        file_map["apps_reviews.csv"],
        file_map["apps_info.csv"],
        id_col="app_id",
        name_col="app_name"
    )
    apps["type"] = "app"
    parts.append(apps)

if not parts:
    raise ValueError("Could not find expected csv files! Check file list.")

df = pd.concat(parts, ignore_index=True)

# SAMPLING
SAMPLE_SIZE = 50_000
if len(df) > SAMPLE_SIZE:
    print(f"Total rows {len(df)}. Sampling {SAMPLE_SIZE}...")
    df = df.sample(n=SAMPLE_SIZE, random_state=42)

print("Final Shape:", df.shape)
df.head()

Loading games_reviews.csv...
Loading games_info.csv...
Loading apps_reviews.csv...
Loading apps_info.csv...
Total rows 1049870. Sampling 50000...
Final Shape: (50000, 14)


,product_id,review_text,review_score,review_date,helpful_count,product_name,description,score,ratings_count,downloads,content_rating,section,categories,type
573804,332,Reminds me of Ant Leigon (one of the GOAT game...,4,2025-03-29,0,Dark War Survival,Gear up and brace yourself for the ultimate te...,4.3,534000,10000000,Everyone 10+,Zombie games,"#5 top grossing strategy, Action, Action-adven...",game
527441,268,I've updated to 4 stars because I beat the lev...,4,2023-12-19,0,Battle Strategy: Tower Defense,Battle Strategy: Tower Defense – Epic WWII TD ...,4.5,95600,1000000,Everyone,Strategy games,"Strategy, Tower defense, Casual, Single player...",game
692523,14,Ordered ice cream via the app and was advised ...,1,2023-09-19,1,Dairy Queen® Food & Treats,Life's Sweet with the DQ® App!Satisfy your cra...,4.5,66100,5000000,Everyone,Popular apps,Food & Drink,app
633047,8,I just love it! So easy to make an order!! Kus...,4,2017-08-18,0,Wingstop,We’re making it easier than ever to bring our ...,4.8,136000,1000000,Everyone,Food and drink apps,Food & Drink,app
958644,174,Nice but not enough people on it,3,2025-01-16,0,Bluesky,Bluesky is THE NEW SOCIAL NETWORK for people w...,3.9,44200,10000000,Mature 17+,Popular apps,Social,app


## 5. Inspect columns

In [6]:
print(df.columns.tolist())
df.info()
df.sample(5, random_state=42)

['product_id', 'review_text', 'review_score', 'review_date', 'helpful_count', 'product_name', 'description', 'score', 'ratings_count', 'downloads', 'content_rating', 'section', 'categories', 'type']
<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 573804 to 539493
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      50000 non-null  int64  
 1   review_text     50000 non-null  object 
 2   review_score    50000 non-null  int64  
 3   review_date     50000 non-null  object 
 4   helpful_count   50000 non-null  int64  
 5   product_name    50000 non-null  object 
 6   description     50000 non-null  object 
 7   score           50000 non-null  float64
 8   ratings_count   50000 non-null  int64  
 9   downloads       50000 non-null  int64  
 10  content_rating  50000 non-null  object 
 11  section         50000 non-null  object 
 12  categories      50000 non-null  object 
 13  type       

,product_id,review_text,review_score,review_date,helpful_count,product_name,description,score,ratings_count,downloads,content_rating,section,categories,type
314201,146,It's entertaining educational and really surpr...,5,2024-06-06,35,Religion Inc. The game god sim,Religion inc – is a simulator of creating a re...,4.3,107000,5000000,Teen,Explore games on PC,"Simulation, Casual, Single player, Stylized, O...",game
189845,113,"I love this with a burning passion, maybe make...",5,2021-09-08,0,Ronin: The Last Samurai,"An era of warfare in Japan, death and betrayal...",4.6,341000,10000000,Teen,Explore games on PC,"Role Playing, Action Role-Playing, Single play...",game
349507,156,Game advertises gold bars for ads but never de...,3,2025-04-13,16,Golf King - World Tour,Realistic Multiplayer Golf at your fingertips!...,4.6,656000,10000000,Everyone,Explore games on PC,"Sports, Golf, Casual, Multiplayer, Competitive...",game
946617,141,I deleted mobius ff game data on my google pla...,2,2018-11-02,83,Google Play Games,Games are more fun with the Google Play Games ...,4.1,13300000,5000000000,Teen,Popular apps,Entertainment,app
233010,125,I used to like this game but I noticed that pe...,2,2020-06-20,2,Magic Rush: Heroes,War is coming – Will you be ready? Summon hero...,4.2,1130000,10000000,Teen,Explore games on PC,"Role Playing, Tactical, Casual, Multiplayer, C...",game


## 6. Column Mapping (Already Standardized)

In [7]:
# Since we standardized earlier, we just map directly
col_id      = "product_id"
col_name    = "product_name"

# Helper to find others if names vary slightly, though in this dataset they seem consistent
def first_existing(df, options):
    cols = {c.lower(): c for c in df.columns}
    for opt in options:
        if opt.lower() in cols:
            return cols[opt.lower()]
    return None

col_text    = first_existing(df, ["review_text", "text", "content", "review"])
col_rating  = first_existing(df, ["review_score", "rating", "score", "stars"])
col_date    = first_existing(df, ["review_date", "date", "timestamp", "time", "at"])
col_helpful = first_existing(df, ["helpful_count", "helpful", "likes", "thumbs_up"])
col_cat     = first_existing(df, ["section", "categories", "category", "genre"])

print("Mapped Columns:")
print("  product_id:", col_id)
print("  product_name:", col_name)
print("  review_text:", col_text)
print("  review_score:", col_rating)
print("  cat/section:", col_cat)

Mapped Columns:
  product_id: product_id
  product_name: product_name
  review_text: review_text
  review_score: review_score
  cat/section: section


## 8. Clean + Standardize (Vectorized)

In [8]:
# Create fresh output DataFrame
out = pd.DataFrame()

# 1. Copy standardized columns
out["product_id"]   = df[col_id].astype(str) if col_id else "unknown"
out["product_name"] = df[col_name].astype(str) if col_name else "unknown"
if "type" in df.columns:
    out["type"] = df["type"]

# 2. Vectorized Text Cleaning
if col_text:
    raw_text = df[col_text].fillna("").astype(str)
    raw_text = raw_text.str.replace(r"https?://\S+|www\.\S+", " ", regex=True)
    raw_text = raw_text.str.replace(r"[\n\r]", " ", regex=True)
    raw_text = raw_text.str.replace(r"\s+", " ", regex=True).str.strip()
    out["review_text"] = raw_text
else:
    out["review_text"] = ""

# 3. Numeric conversions
if col_rating:
    out["rating"] = pd.to_numeric(df[col_rating], errors="coerce")
else:
    out["rating"] = np.nan

if col_helpful:
    out["helpful_count"] = pd.to_numeric(df[col_helpful], errors="coerce").fillna(0).astype(int)
else:
    out["helpful_count"] = 0

if col_date:
    out["timestamp"] = pd.to_datetime(df[col_date], errors="coerce", utc=True)
else:
    out["timestamp"] = pd.NaT

if col_cat:
    out["category"] = df[col_cat].astype(str)
else:
    out["category"] = ""

# 4. Filtering
out = out[out["review_text"].str.len() >= 5].copy()
out = out.drop_duplicates(subset=["product_id", "review_text"], keep="first")

# 5. Create Review ID
hash_input = (
    out["product_id"].fillna("unknown") + "::" +
    out["review_text"].str.slice(0, 120) + "::" +
    out["timestamp"].astype(str)
)
out["review_id"] = hash_input.apply(lambda s: str(abs(hash(s))))

print("Cleaned reviews:", out.shape)
out.head()

Cleaned reviews: (49826, 9)


,product_id,product_name,type,review_text,rating,helpful_count,timestamp,category,review_id
573804,332,Dark War Survival,game,Reminds me of Ant Leigon (one of the GOAT game...,4,0,2025-03-29 00:00:00+00:00,Zombie games,3121352841848221248
527441,268,Battle Strategy: Tower Defense,game,I've updated to 4 stars because I beat the lev...,4,0,2023-12-19 00:00:00+00:00,Strategy games,7695363490180194395
692523,14,Dairy Queen® Food & Treats,app,Ordered ice cream via the app and was advised ...,1,1,2023-09-19 00:00:00+00:00,Popular apps,5138878453819103198
633047,8,Wingstop,app,I just love it! So easy to make an order!! Kus...,4,0,2017-08-18 00:00:00+00:00,Food and drink apps,2617221958169505988
958644,174,Bluesky,app,Nice but not enough people on it,3,0,2025-01-16 00:00:00+00:00,Popular apps,5121406095307252313


## 9. Save Processed Reviews

In [9]:
output_dir = Path("data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

out.to_parquet(output_dir / "sentio_plus_reviews.parquet", index=False)
out.to_csv(output_dir / "sentio_plus_reviews.csv", index=False)

print("Saved processed data to:", output_dir)

Saved processed data to: data/processed


## 10. Build Product Scores (Vectorized Aggregation)

In [10]:
# Filter valid ratings
ratings = out.dropna(subset=["rating"])
ratings = ratings[ratings["rating"].between(1, 5)].copy()

# 1. Calculate weighted sum for average
ratings["weight"] = 1 + ratings["helpful_count"]
ratings["weighted_score"] = ratings["rating"] * ratings["weight"]

# 2. GroupBy once
grouped = ratings.groupby(["product_id", "product_name"])

agg = grouped.agg(
    rating_count=("rating", "size"),
    avg_rating=("rating", "mean"),
    helpful_votes=("helpful_count", "sum"),
    total_weighted_score=("weighted_score", "sum"),
    total_weight=("weight", "sum")
)

# 3. Calculate weighted average vectorially
agg["weighted_avg_rating"] = agg["total_weighted_score"] / agg["total_weight"]

# 4. Star Counts (Pivot)
ratings["star"] = ratings["rating"].round().astype(int)
star_counts = (
    ratings.groupby(["product_id", "product_name", "star"])
    .size()
    .unstack(fill_value=0)
)

# Ensure columns 1-5 exist
for s in range(1, 6):
    if s not in star_counts.columns:
        star_counts[s] = 0

star_counts = star_counts.rename(columns={s: f"star_{s}_count" for s in range(1, 6)})

# Join everything
product_scores = agg.join(star_counts, on=["product_id", "product_name"], how="left").reset_index()

product_scores = product_scores.drop(columns=["total_weighted_score", "total_weight"])
product_scores["avg_rating"] = product_scores["avg_rating"].round(2)
product_scores["weighted_avg_rating"] = product_scores["weighted_avg_rating"].round(2)

print("Product scores shape:", product_scores.shape)
product_scores.sort_values("rating_count", ascending=False).head(10)

Product scores shape: (245, 11)


,product_id,product_name,rating_count,avg_rating,helpful_votes,weighted_avg_rating,star_1_count,star_2_count,star_3_count,star_4_count,star_5_count
1,1,Western Union Send Money Now,1009,3.08,2658,3.07,248,126,169,231,235
32,13,Scanner Radio - Police Scanner,879,3.26,3944,3.34,192,71,156,234,226
241,97,"Nike: Shoes, Apparel & Stories",758,3.01,8263,3.11,145,163,144,152,154
83,17,Yahoo Mail,752,2.98,8865,2.92,157,140,156,157,142
41,137,Realtor.com Real Estate & Rent,750,2.98,2045,2.87,156,142,158,146,148
87,176,Domino's Pizza USA,722,2.95,1386,2.82,156,140,148,139,139
222,56,Google Calendar,720,3.03,11644,3.09,134,150,140,153,143
225,58,Subway®,716,2.97,2571,2.73,163,126,147,131,149
220,55,Workday,705,3.03,684,2.56,150,141,114,138,162
86,175,Alibaba.com - B2B marketplace,704,2.91,101726,3.43,154,155,133,127,135


## 11. Save Product Scores

In [11]:
product_scores.to_parquet(output_dir / "sentio_plus_product_scores.parquet", index=False)
product_scores.to_csv(output_dir / "sentio_plus_product_scores.csv", index=False)

print("Saved product scores.")

Saved product scores.


## 12. Upsert to ChromaDB

In [12]:
import chromadb

client = chromadb.PersistentClient(path="chroma_store")
collection = client.get_or_create_collection(name="sentio_plus_reviews")

# Convert DataFrame to lists for Chroma
chroma_df = out.dropna(subset=["review_text", "review_id"])

ids = chroma_df["review_id"].astype(str).tolist()
documents = chroma_df["review_text"].tolist()

metadatas = [
    {
        "product_id": str(row["product_id"]),
        "product_name": str(row["product_name"]),
        "rating": float(row["rating"]) if pd.notna(row["rating"]) else -1.0,
        "helpful_count": int(row["helpful_count"]),
        "category": str(row["category"]),
        "type": str(row["type"]) if "type" in row and pd.notna(row["type"]) else "unknown",
        "timestamp": row["timestamp"].isoformat() if pd.notna(row["timestamp"]) else ""
    }
    for _, row in chroma_df.iterrows()
]

BATCH_SIZE = 2000
print(f"Upserting {len(ids)} documents in batches of {BATCH_SIZE}...")

for i in range(0, len(ids), BATCH_SIZE):
    sys_ids = ids[i:i+BATCH_SIZE]
    sys_docs = documents[i:i+BATCH_SIZE]
    sys_metas = metadatas[i:i+BATCH_SIZE]
    
    collection.upsert(
        ids=sys_ids,
        documents=sys_docs,
        metadatas=sys_metas
    )
    if i % 10000 == 0:
        print(f"  Processed {i}/{len(ids)}")

print("Done upserting to ChromaDB.")

Upserting 49826 documents in batches of 2000...
  Processed 0/49826
  Processed 10000/49826
  Processed 20000/49826
  Processed 30000/49826
  Processed 40000/49826
Done upserting to ChromaDB.


## 13. Example Queries

In [13]:
results = collection.query(
    query_texts=["app crashes on startup"],
    n_results=3
)

for i, doc in enumerate(results['documents'][0]):
    print(f"Result {i+1}: {doc[:100]}...")
    print(f"Metadata: {results['metadatas'][0][i]}")
    print("-" * 40)

Result 1: App crash all the time...
Metadata: {'type': 'app', 'category': 'Budgeting tools', 'timestamp': '2022-07-08T00:00:00+00:00', 'product_name': 'Western Union Send Money Now', 'product_id': '1', 'rating': 1.0, 'helpful_count': 0}
----------------------------------------
Result 2: App crash all the time...
Metadata: {'type': 'app', 'product_id': '1', 'product_name': 'Western Union Send Money Now', 'helpful_count': 0, 'timestamp': '2022-07-08T00:00:00+00:00', 'category': 'Budgeting tools', 'rating': 1.0}
----------------------------------------
Result 3: The app crashes all the time...
Metadata: {'type': 'app', 'product_id': '55', 'category': 'Popular apps', 'helpful_count': 1, 'rating': 2.0, 'product_name': 'Workday', 'timestamp': '2023-08-08T00:00:00+00:00'}
----------------------------------------


## 14. Rerank results using helpful_count

In [14]:
import math

def rerank_by_helpful(query_text, where=None, n_retrieve=50, n_return=10):
    raw = collection.query(
        query_texts=[query_text],
        n_results=n_retrieve,
        where=where
    )
    docs = raw["documents"][0]
    metas = raw["metadatas"][0]
    dists = raw.get("distances", [None])[0]  # if available

    # If distances exist: smaller distance = better
    # We'll convert to a similarity-like score for combining.
    scored = []
    for i in range(len(docs)):
        helpful = metas[i].get("helpful_count", 0) or 0
        helpful_boost = math.log1p(helpful)

        if dists is not None:
            base = 1.0 / (1.0 + float(dists[i]))  # simple invert
        else:
            base = 1.0

        final = base * (1.0 + 0.15 * helpful_boost)  # tune 0.15 as you like
        scored.append((final, docs[i], metas[i]))

    scored.sort(key=lambda x: x[0], reverse=True)
    return scored[:n_return]

top = rerank_by_helpful(
    "latest update causes crashes",
    where={"product_id": out["product_id"].iloc[0]},
    n_retrieve=50,
    n_return=10
)

top[:3]


[(0.6396456970723935,
  "Demoting my review from 4 stars to 1 star, and wish I could give none! Game was fine for a while, then started not loading any chats in game at all. After clearing game cache & restarting my phone it didn't help. So I uninstalled & reinstalled game, twice! Also checked allowed permissions. Still can't get past 20 percent loading screen network connection error. Tried over & over & now can't even play the game at all. Been playing about 2 months just fine. Very disappointed.😠",
  {'timestamp': '2025-02-20T00:00:00+00:00',
   'helpful_count': 36,
   'category': '',
   'product_name': 'unknown',
   'rating': 1.0,
   'product_id': '332'}),
 (0.588402848422127,
  "Got too log in one time an since then every time I've try too log back in I always get a * connection error* an I know there is nothing wrong with my Internet connection. (UPDATE) been over a week since I've downloaded this game an still can't play on wifi.... Game only works if I use my data an I refuse t